### Prerequisites
Before starting this notebook, make sure you've worked through `exploring_files.ipynb`! This notebook assumes you've understood what was presented there.

We'll use the same packages we used in the previous notebook.

In [ ]:
import uproot
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

We're also going to perform some fits to the data, so we'll need an additional package called `iminuit`. To fit a function to data, we numerically minimize some cost function that quantifies the goodness of fit. `iminuit` provides python access to the decades-old MINUIT minimization package. We'll also use some convenient functions from the `scipy` package.

In [ ]:
import iminuit

Now we can get our data and calculate the masses we'll need:

In [ ]:
f = uproot.open('/share/lazy/tboettch/tutorial/B2HHH_MagnetDown.root')
t = f['DecayTree']
arrs = t.arrays()

m_k = 497.611

h1_px = arrs['H1_PX']
h1_py = arrs['H1_PY']
h1_pz = arrs['H1_PZ']
h1_p = np.sqrt(h1_px**2 + h1_py**2 + h1_pz**2)
h1_e = np.sqrt(h1_p**2 + m_k**2)
h1_p4 = np.array([h1_px, h1_py, h1_pz, h1_e])

h2_px = arrs['H2_PX']
h2_py = arrs['H2_PY']
h2_pz = arrs['H2_PZ']
h2_p = np.sqrt(h2_px**2 + h2_py**2 + h2_pz**2)
h2_e = np.sqrt(h2_p**2 + m_k**2)
h2_p4 = np.array([h2_px, h2_py, h2_pz, h2_e])

h3_px = arrs['H3_PX']
h3_py = arrs['H3_PY']
h3_pz = arrs['H3_PZ']
h3_p = np.sqrt(h3_px**2 + h3_py**2 + h3_pz**2)
h3_e = np.sqrt(h3_p**2 + m_k**2)
h3_p4 = np.array([h3_px, h3_py, h3_pz, h3_e])

b_p4 = h1_p4 + h2_p4 + h3_p4
b_m = np.sqrt(b_p4[3]**2 - b_p4[0]**2 - b_p4[1]**2 - b_p4[2]**2)
arrs['B_M'] = b_m

h12_p4 = h1_p4 + h2_p4
h12_m = np.sqrt(h12_p4[3]**2 - h12_p4[0]**2 - h12_p4[1]**2 - h12_p4[2]**2)
arrs['H12_M'] = h12_m

h13_p4 = h1_p4 + h3_p4
h13_m = np.sqrt(h13_p4[3]**2 - h13_p4[0]**2 - h13_p4[1]**2 - h13_p4[2]**2)
arrs['H13_M'] = h13_m

Now we'll apply a selection to get us a clean sample.

In [ ]:
sel_arrs = arrs[
    (arrs['H1_ProbK']>0.5) & (arrs['H2_ProbK']>0.5) & (arrs['H3_ProbK']>0.5) #&
    # (arrs['H1_ProbPi']<0.5) & (arrs['H2_ProbPi']<0.5) & (arrs['H3_ProbPi']<0.5) &
    # (arrs['H1_isMuon']==0) & (arrs['H2_isMuon']==0) & (arrs['H3_isMuon']==0)
]

plt.clf()
ax = plt.gca()
ax.hist(sel_arrs['B_M'], range=(5100, 5500), bins=100)
ax.set_xlim(5100, 5500)
plt.show()

### Searching for $CP$ violation.
This looks like a pretty clean sample! One of the goals of looking at a sample like this is to try to study matter-antimatter asymmetries, or $CP$ violation. One signature of $CP$ violation would be an asymmetry of $B^+\to K^+K^-K^+$ and $B^-\to K^-K^+K^-$. Let's see if we can see $CP$ violation in this sample. First, let's look at the intermediate masses `H12_M` and `H23_M`.

In [ ]:
plt.clf()
ax = plt.gca()
ax.hist(sel_arrs['H12_M'], range=(980, 4700), bins=100)
ax.set_xlim(980, 4700)
plt.show()

plt.clf()
ax = plt.gca()
ax.hist(sel_arrs['H13_M'], range=(980, 4700), bins=100)
ax.set_xlim(980, 4700)
plt.show()

These distributions have a lot of structure. First, we have a large peak at the low edge of the mass spectrum. This is the $\phi$ meson, which almost always decays to $K^+K^-$. At around $1800-2000~{\rm MeV}$, we have $D$ mesons decaying to $K^+K^-$. Then at around $3400~{\rm MeV}$, we have a $J/\psi$ peak. The $J/\psi$ is a meson made up of a charm-anticharm quark pair. It has a mass of about $3100~{\rm MeV}$ and often decays to muons. If we misidentify muons as kaons, we end up with a peak at higher mass. The underlying physics of $B$ mesons decaying to particles containing charm quarks is different from that of so-called "charmless" $B$ decays, so we'll veto the $D$ and $J/\psi$ peaks.

In [ ]:
sel_arrs = sel_arrs[
    # Veto the D peaks.
    ((sel_arrs['H12_M']<1830) | (sel_arrs['H12_M']>1910)) &
    ((sel_arrs['H13_M']<1830) | (sel_arrs['H13_M']>1910)) &
    # Veto the J/psi peaks. Note the "correct" way to do this would be to
    # replace the kaon mass with the muon mass and recalculate the J/psi mass,
    # but this eye-balled approach will be good enough for now.
    ((sel_arrs['H12_M']<3350) | (sel_arrs['H12_M']>3450)) &
    ((sel_arrs['H13_M']<3350) | (sel_arrs['H13_M']>3450))
]

plt.clf()
ax = plt.gca()
ax.hist(sel_arrs['H12_M'], range=(980, 4700), bins=100)
plt.show()

plt.clf()
ax = plt.gca()
ax.hist(sel_arrs['H13_M'], range=(980, 4700), bins=100)
plt.show()

There are still some structures remaining, but we'll ignore them for now.

Now, to look for $CP$ violation, we need to create $B^+$ and $B^-$ samples.

In [ ]:
sel_arrs['B_Charge'] = sel_arrs['H1_Charge'] + sel_arrs['H2_Charge'] + sel_arrs['H3_Charge']

plt.clf()
ax = plt.gca()
ax.hist(sel_arrs[sel_arrs['B_Charge']>0]['B_M'], range=(5100, 5500), bins=100, histtype='step', label=r'$B^+$')
ax.hist(sel_arrs[sel_arrs['B_Charge']<0]['B_M'], range=(5100, 5500), bins=100, histtype='step', label=r'$B^-$')
ax.set_xlim(5100, 5500)
ax.legend()
plt.show()

We can start to see hints of an asymmetry. But to know for sure, we need to know the yields of $B^+$ and $B^-$ candidates. We can do this by performing a fit.

### Modeling signal and background
The first step in performing a fit is to come up with a model for the signal and background. Our signal looks like a roughly symmetric peak centered at the $B$ mass, so we'll model this using a Gaussian. The background appears to be slowly decreasing, so we'll try an exponential. 

A Gaussian function has two parameters usually referred to as $\mu$ and $\sigma$. $\mu$ is the location of the center of the peak, and $\sigma$ is the width. In `scipy`, the Gaussian function is called `norm`, and $\mu$ and $\sigma$ are called `loc` and `scale`, respectively. The exponential will also have parameters `loc` and `scale`. We'll use `loc` to shift the exponential to our fit range, and `scale` will control how quickly the exponential decreases. We'll use the variable `a` for the `scale`. In both cases, we'll be accessing the probability density functions (PDFs) of these distributions.

#### Note: guessing a fit model
In real life, your fit model should be informed by your knowledge of the underlying physics. Here we've chosen an exponential to model the background, but in reality this is a poor choice. This is because we know that the background at low mass is due to partially reconstructed $B$ decays and $B$ decays where we've misidentified pions as kaons. These mis-ID and partially reconstructed backgrounds have shapes that differ from an exponential. If we wanted to perform the best possible measurement, we'd carefully model these backgrounds. But an exponential will work pretty well and anything more sophisticated is beyond the scope of this tutorial.

In [ ]:
from scipy.stats import norm
from scipy.stats import expon

def pdf(m, n_s, n_b, mu, sigma, a):
    return n_s*norm.pdf(m, mu, sigma) + n_b*expon.pdf(m, loc=5100, scale=a)

Let's start by trying to minimize the $\chi^2$. The $\chi^2$ is defined as:
$$
\chi^2(\vec{\theta})=\sum_{i}\frac{(y_i-f(x_i,\vec{\theta}))^2}{\sigma_i^2}
$$
where $\vec{\theta}$ is the vector of fit parameters (e.g. `n_s`, `n_b`, `mu`, `sigma`, and `a` above) and $f$ is the fit function. For a histogram, we can start by assuming $x_i$ is the center of bin $i$, and $y_i$ is the bin contents. Then $\sigma_i$ will be $\sqrt{y_i}$. We can get these values using numpy. Let's define $\chi^2$ functions for the $B^+$ and $B^-$ below.

#### Note: Limitations of the $\chi^2$
The $\chi^2$ is probably the most commonly used cost function for fitting across all of science, but it's really just a special case of a more general cost function called the likelihood $\mathcal{L}$. The $\chi^2$ doesn't work well if our bins have few events, and it becomes infinite if any of the bins are empty. It works well enough for this example, but for a real analysis you'd want to calculate a likelihood that takes into account the fact that the contents of each bin comes from a Poisson distribution. If you're interested, the `iminuit` documentaion has more details about different cost functions:

https://iminuit.readthedocs.io/en/stable/notebooks/cost_functions.html

In [ ]:
# Get the bin contents and bin edges for each sample.
y_bp, edges = np.histogram(sel_arrs[sel_arrs['B_Charge']>0]['B_M'], range=(5100, 5500), bins=100)
y_bm, _ = np.histogram(sel_arrs[sel_arrs['B_Charge']<0]['B_M'], range=(5100, 5500), bins=100)
# We'll need the y values to be numpy arrays for the rest of our calculations.
y_bp = np.array(y_bp)
y_bm = np.array(y_bm)
edges = np.array(edges)
# Calculate the bin centers.
x = 0.5*(edges[:-1] + edges[1:])
# We'll need the bin widths to turn the PDF into bin contents.
widths = edges[1:] - edges[:-1]
# Caclaulte the uncertainties.
err_bp = np.sqrt(y_bp)
err_bm = np.sqrt(y_bm)

# Define the chi2.
def chi2_bp(n_s, n_b, mu, sigma, a):
    ressq = (y_bp - widths*pdf(x, n_s, n_b, mu, sigma, a))**2/err_bp**2
    return ressq.sum()

# We've defined this chi2 function by storing the data itself in global
# variables, but we could also create a class that stores the data and
# calculates the chi2. For more complex situations, this ends up being much more
# convenient. Give it a shot!


Once we've defined the $\chi^2$, we can use `iminuit` to minimize it. Let's try it.

In [ ]:
# We need some initial guesses for the parameters

init_pars = [
    y_bp.sum()*0.95, # Guess we have about 95% signal and 5% background.
    y_bp.sum()*0.05,
    5280., # This is roughly the B mass.
    50., # We know LHCb's mass resolution should be ~1%.
    400. # The exponential doesn't fall off too fast, so let's start by guessing the scale is the fit range.
]

# Create the minimizer
minimizer_bp = iminuit.Minuit(chi2_bp, *init_pars)

# We need to set some limits on the parameters, or else the minimization could
# try something crazy.
limits = [
    (0, np.inf), # Number of signal and background need to be >= 0.
    (0, np.inf),
    (5250, 5310), # We roughly know the position of the B peak
    (1., 200.), # Don't let the resolution be too large or too small.
    (1., np.inf) # The exponential doesn't fall off too fast, so the scale should be pretty large.
]
minimizer_bp.limits = limits

# Now we can perform the minimization.
minimizer_bp.migrad()


The minimizer says our fit converged to a valid minimum. We can get an idea of
the quality of our fit by looking at the value of `FCN`, which is the $\chi^2$
we're minimizing. We ended up with a final $\chi^2=104.2$. We typically assess the fit quality by looking at the $\chi^2$ per degrees of freedom, or $\chi^2/{\rm ndf}$. The number of degrees of freedom will be the number of data points minus the number of free parameters. In this case, ${\rm ndf}=100-5=95$. A $\chi^2/{\rm ndf}$ of about $1$ indicates a good fit, while a $\chi^2/{\rm ndf}$ of much less than $1$ indicates the data has been over fit: the fit model has too much freedom to describe small statistical fluctuations in the data. A large $\chi^2/{\rm ndf}$ indicates that the fit model doesn't provide a good description of the data. Our $\chi^2/{\rm ndf}$ is about $1.1$, which is pretty good.

The $\chi^2/{\rm ndf}$ is a nice tool, but nothing beats visually inspecting the fit results. Once we perform a fit, we can access the fit values and errors using `minimizer_bp.values` and `minimizer_bp.errors`, respectively. Let's try drawing the fit function with the best fit parameters.

In [ ]:
fit_xs = np.linspace(5100, 5500, 500)
fit_ys = pdf(fit_xs, *minimizer_bp.values)

plt.clf()
ax = plt.gca()
ax.errorbar(x, y_bp, yerr=err_bp, fmt='.', color='black')
# Remember, we need to multiply by the bin width to turn the PDF value into a yield.
ax.plot(fit_xs, widths[0]*fit_ys)
ax.set_ylim(0)
ax.set_xlim(5100, 5500)
plt.show()

This looks like a pretty sensible fit! Now if we want to look for $CP$ violation, we have to repeat this with the $B^-$ sample. Try to define a $\chi^2$ for the $B^-$ sample and minimize it to determine `n_s` for both samples.

In [ ]:
# Repeat the fit but for the B- sample.

Once you have the `n_s` for each sample, you can calculate the $CP$ asymmetry for this decay:
$$
A_{CP}=\frac{n^-_s-n^+_s}{n^-_s+n^+_s}
$$
Use your fit results to calculate the $CP$ asymmetry, and propagate the uncertainties from the fit result through the calculation to determine the statistical uncertainty on the asymmetry. How large is the uncertainty, and is it significantly different from 0?

#### Note: Correlated uncertainties
You need to be careful with the uncertainty propagation, because the statistical uncertainties on the numerator and denominator are fully correlated. This means the final uncertainty on $A_{CP}$ is a little smaller than what you'd get if you just added the uncertainties in quadrature.

In [ ]:
# Calculate the resulting CP asymmetry.

Congratulations! You've finished your first measurement of $CP$ violation in $B$ meson decays. You can compare your result to the published LHCb result here:

https://cds.cern.ch/record/1751517/files/PhysRevD.90.112004.pdf

How does your result compare to the published measurement? Are the two measurements consistent? Take a quick look at the paper to get an idea of all of the effects we ignored during this exercise (e.g. production and detection asymmetries). While these effects may be small, they're extremely important to understand for high-precision measurements. When you work on an actual measurement, the vast majority of your time will be spent on these "minor" details, but you'll study them using the same basic tools you used here.

### Next steps
Now that you know what you're doing, try repeating this analysis for the $B^{\pm}\to\pi^{\pm}\pi^{\mp}\pi^{\pm}$ channel. This channel has a lot more background, but the overall steps are the same. Recalculate the four momenta using the $\pi$ mass, develop an event selection, and try extracting the $CP$ asymmetry.

In [1]:
# Insert B -> 3pi analysis here.